In [ ]:
import pandas as pd
df = pd.read_csv('weibo_senti_100k.csv')
df

,content
0,【俄乌进入谈判前“至暗时刻”？泽连斯基最新言论已暗含变化？】俄罗斯国防部19日证实，乌克兰当...
1,【废除法规 解雇员工 #马斯克公布大规模政府裁员计划#】据央视新闻消息，当地时间11月20日...
2,11月20日，乌克兰使用英国“风暴阴影”巡航导弹，攻击了俄罗斯境内目标。按照俄罗斯国防部的战...
3,牛市结束，市场回归平静。最终资金还是要选择业绩股，低价股，但是市场上真正的低价股还有多少？新...
4,马斯克新论:美国就是新罗马。#马斯克# #特朗普新任命# ​
...,...
22179,巴勒斯坦“卡桑旅”（哈马斯）发布新视频，在加沙地区中部走廊狙击一名以色列士兵，使用的是AM-...
22180,#巴以冲突##看见具体的巴勒斯坦人# 巴勒斯坦记者Hossam·Shabat和加沙平民近照 ​
22181,#巴以冲突# 圣城新闻网：以色列军车和直升机在加沙地带北部贝特拉希耶西部地区猛烈扫射，同时还...
22182,11月15日，在德方倡议下，普京与朔尔茨进行了自2022年12月以来的首次电话会谈。双方就乌...


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='weibo_senti_100k.csv', encoding='utf-8')
data = loader.load()


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(data)


In [19]:
len(all_splits)


22184

In [17]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings # type: ignore
from langchain_community.vectorstores import FAISS # type: ignore
from datetime import datetime

# 1. 设置嵌入模型
model_name = "BAAI/bge-large-zh-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
bgeEmbeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)


In [20]:
# 2. 构建向量数据库-
vector = FAISS.from_documents(all_splits, bgeEmbeddings)

In [21]:

# 3. 保存向量数据库到本地
vector.save_local("weibo_2.0")  # 保存路径为 "weibo""zhongyi""zhongyi-all""dfcf"

In [ ]:
import os
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from zhipuai import ZhipuAI

# 设置 ZhipuAI API Key 为环境变量
os.environ["ZHIPUAI_API_KEY"] = "you_key_in_there"

# 初始化 ZhipuAI 客户端
client = ZhipuAI()

# 1. 设置嵌入模型（需与保存向量数据库时一致）
model_name = "BAAI/bge-large-zh-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
bgeEmbeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# 2. 载入本地保存的向量数据库
db_path = "weibo/"  # 向量数据库的本地保存路径
new_db = FAISS.load_local(db_path, bgeEmbeddings, allow_dangerous_deserialization=True)

# 3. 执行查询从数据库中检索相关文档
query = "当前局势"
docs = new_db.similarity_search(query)

# 4. 提取检索结果内容作为上下文
if docs:
    context = "\n\n".join([doc.page_content for doc in docs])
else:
    context = "未找到相关文档内容。"

# 5. 调用 ZhipuAI 的 API 生成回答
optimized_prompt = (
    "请根据本地知识库，回答以下问题，并始终用中文作答。\n\n"
    "### 回答规则：\n"
    "1. 优先使用本地微博评论知识库中的信息进行回答。\n"
    "2. 如果本地微博评论知识库中的信息不足，请补充使用网络搜索结果。\n"
    "3. 如果无法从本地微博评论知识库或网络搜索中找到相关信息，请回答：“此信息在本地微博评论知识库或网络搜索结果中均不可用。”\n\n"
    "### 提供的本地微博评论知识库：\n"
    "\"\"\"\n{context}\n\"\"\"\n\n"
    "### 问题：\n"
    '"""\n{query}\n"""\n\n'
    "### 回答要求：\n"
    "1. 请直接提供最终答案。\n"
    "2. 明确指出答案的来源：“微博评论知识库”、“网络搜索”或“两者结合”。\n"
    "3. 如果回答中包含对评论内容的分析或解释，请提供简要的说明。\n"
    "4. 确保回答内容简洁明了，符合网络交流的表述习惯，避免冗长。\n\n"
    "### 请开始回答："
    "### 举例： \n"
    "问题1： 用户对某明星新电影的评论态度如何？\n"
    "回答1：多数用户持正面态度，认为电影剧情吸引人，演员表现力强。\n"
    "问题2： 网友对某品牌手机最新款式的评价怎样？\n"
    "回答2：评价褒贬不一，部分用户认为新款手机设计新颖，性能提升明显，但也有用户反映价格偏高，性价比不如预期。\n"
)


response = client.chat.completions.create(
    model="glm-4-long",
    messages=[
        {
            "role": "user",
            "content": optimized_prompt.format(context=context, query=query),
        }
    ],
    stream=True,
)

# 6. 拼接响应的内容
result = ""
for chunk in response:
    content = chunk.choices[0].delta.content
    if content:
        result += content

# 7. 打印最终回答结果
print(result)

当前局势包括国际关系和国内经济状况。
1. 国际关系方面，根据本地微博评论知识库中的信息，当前国际局势复杂，西方国家在某些问题上可能面临挑战。这表明在国际政治舞台上，不同国家和地区的利益和立场存在分歧，需要通过对话和协商来解决问题。
2. 国内经济状况方面，本地微博评论知识库显示，我国经济正在逐步恢复，股市和楼市已经基本稳定。这说明我国政府采取了一系列有效措施，保持了经济的稳定增长，为各行业逐步恢复提供了有力支撑。

综上所述，当前局势在国际关系上面临挑战，但在国内经济上保持稳定增长。这一分析主要来源于本地微博评论知识库。
